In [1]:
import numpy as np
from egomotion_utils.offline_egomotion_utils import retrieve_sequence_data, preprocess_data, \
                                                    fx, hx, mean_x, subtract_x, add_x, normalize_angle, \
                                                    forward_ukf, backward_ukf, backward_smoothing_ukf, \
                                                    postprocess_data, write_data_to_json_file
                                                    
from unscented_kalman_filter import UKF_gen
                                                    
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [6]:
import os
from tqdm import tqdm

# #record_dir = "/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-30/Cobra/20231030-114445/slices/20231030-114445-00.09.15-00.09.30@Cobra/"
# record_dir = "/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-30/Cobra/20231030-105515/slices/20231030-105515-00.11.15-00.11.30@Cobra"            
# #record_dir = '/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-30/Cobra/20231030-150910/slices/20231030-150910-00.09.30-00.09.45@Cobra'
# #record_dir = '/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-30/Cobra/20231030-150910/slices/20231030-150910-01.31.45-01.32.00@Cobra'
# #record_dir ='/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-30/Cobra/20231030-150910/slices/20231030-150910-01.32.30-01.32.45@Cobra'
# record_dir = '/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-25/Nevada/20231025-104707/slices/20231025-104707-00.00.15-00.00.30@Nevada'


root_dir = "/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-13/Sogun/20231013-095705/slices"
list_record_dir = sorted(os.listdir(root_dir))

for rd in tqdm(list_record_dir[125:]):
     
     record_dir = os.path.join(root_dir, rd)

     gps_data, veh_data = retrieve_sequence_data(record_dir)

     dt = 0.01
     t_sync, y_meas, u = preprocess_data(gps_data, veh_data, dt)

     x0 = np.zeros((3,1))
     Px0 = 10*np.eye(3)
     Px0[2,2] = 1*(np.pi/2)**2
     #np.where(~np.isnan(y_meas))[0][0]

     ukf = UKF_gen(nx = 3, ny = 2, fx = fx, hx = hx, deltaT = dt, implem='std', alpha=1., beta=0., kappa =0., mean_x = mean_x, subtract_x = subtract_x, add_x = add_x)
     ukf.Q = (((1/3.6)*dt)**2)*np.eye(3)
     ukf.Q[2,2] = ((1*(np.pi/180)*dt)**2)
     ukf.R = (0.005**2)*np.eye(2)

     x_tab1, Px_tab1 = forward_ukf(ukf, x0, Px0, u, y_meas)

     xf = x_tab1[-1]
     Pxf = Px_tab1[-1]

     # orientation and yaw rate reversal for backward pass
     xf[2] = normalize_angle(xf[2]+np.pi)
     ur = np.copy(u)
     ur[:,1] = -u[:,1]

     x_tab2, Px_tab2 = backward_ukf(ukf, xf, Pxf, ur, y_meas)
     x0 = x_tab2[0]
     Px0 = Px_tab2[0]
     # orientation reversal
     x0[2] = normalize_angle(x0[2]+np.pi)
     x_tab3, Px_tab3 = forward_ukf(ukf, x0, Px0, u, y_meas)
     x_tab4, Px_tab4 = backward_smoothing_ukf(ukf, x_tab3, Px_tab3, u)

     RT_ecef_body, lat_lon_alt = postprocess_data(x_tab4, gps_data["lat"][0], gps_data["lon"][0], gps_data["alt"][0])

     write_data_to_json_file(t_sync, RT_ecef_body, f'/shared/home/u364945/odometry/20231218_173949_RecFile_1/egomotion_est_{rd}.json')

    

  0%|          | 0/96 [00:00<?, ?it/s]

 19%|█▉        | 18/96 [2:47:14<12:30:01, 576.94s/it]

In [7]:
import json
import numpy as np
import pymap3d as pm
import matplotlib.pyplot as plt
import os
import gpxpy
import gpxpy.gpx
import glob
from tqdm import tqdm

root_dir = "/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-13/Sogun/20231013-095705/slices"
list_record_dir = sorted(os.listdir(root_dir))

for rd in tqdm(list_record_dir[102:]):
    
    record_dir = os.path.join(root_dir, rd)

    gpx = gpxpy.gpx.GPX()

    # Create first track in our GPX:
    gpx_track1 = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track1)

    # Create first segment in our GPX track 1:
    gpx_segment1 = gpxpy.gpx.GPXTrackSegment()
    gpx_track1.segments.append(gpx_segment1)

    dir = os.listdir(record_dir.split('/slices/')[0] + '/gt/')
    ego_motion_path = record_dir.split('/slices/')[0] + '/gt/' + dir[0] + '/daa/' + record_dir.split('/slices/')[-1] + '/sensor/gnssins/egomotion2.json'
    #ego_motion_path = '/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-30/Cobra/20231030-105515/gt/653059470/daa/20231030-105515-00.11.15-00.11.30@Cobra/sensor/gnssins/egomotion2.json'
    ego_motion_file = open(ego_motion_path)
    ego_motion_data = json.load(ego_motion_file)

    # n_start = np.where(t_sync[0] <= time_host)[0][0]
    # n_end = np.where(t_sync[-1] >= time_host)[0][-1]

    for n, key in enumerate(ego_motion_data):
        
        RT_ecef_body = ego_motion_data[key]['RT_ECEF_body']
        X_ecef_body = np.array(RT_ecef_body)[0:3,3]    
        lat, lon, h = pm.ecef2geodetic(*X_ecef_body)
        # Create points:
        gpx_segment1.points.append(gpxpy.gpx.GPXTrackPoint(lat, lon, h))
    
    f = open(f"/shared/home/u364945/odometry/20231218_173949_RecFile_1/gt_{rd}.gpx", "w")
    f.write(gpx.to_xml())
    f.close()

    gpx = gpxpy.gpx.GPX()

    # Create first track in our GPX:
    gpx_track2 = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track2)

    # Create first segment in our GPX track:
    gpx_segment2 = gpxpy.gpx.GPXTrackSegment()
    gpx_track2.segments.append(gpx_segment2)

    ego_motion_file = open(f"/shared/home/u364945/odometry/20231218_173949_RecFile_1/egomotion_est_{rd}.json")
    ego_motion_data = json.load(ego_motion_file)

    for n, key in enumerate(ego_motion_data):
        
        RT_ecef_body = ego_motion_data[key]['RT_ECEF_body']
        X_ecef_body = np.array(RT_ecef_body)[0:3,3]    
        lat, lon, h = pm.ecef2geodetic(*X_ecef_body)

        # Create points:
        gpx_segment2.points.append(gpxpy.gpx.GPXTrackPoint(lat, lon, h))

    f = open(f"/shared/home/u364945/odometry/20231218_173949_RecFile_1/gt_est_{rd}.gpx", "w")
    f.write(gpx.to_xml())
    f.close()

 35%|███▍      | 51/147 [05:34<10:28,  6.55s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/shared/home/u364945/odometry/20231218_173949_RecFile_1/egomotion_est_20231013-095705-00.41.45-00.42.00@Sogun.json'

In [8]:
import gpxpy
import glob

list_gpx = glob.glob("gt_est*095705*")

gpx_out = gpxpy.gpx.GPX()
gpxs=[]#: List[gpxpy.gpx.GPX] = []

for g in list_gpx:
    gpxs.append(gpxpy.parse(open(g)))
    
for g in gpxs:
    for track in g.tracks:
        gpx_out.tracks.append(track)
    
f = open("/shared/home/u364945/odometry/20231218_173949_RecFile_1/gt_est_095705.gpx", "w")
f.write(gpx_out.to_xml())


    
    

26229700

[]

In [2]:
import os
root_dir = "/data_zfs/private_datasets/aidvs/data/00_daily_testing/2023-10-13/Sogun/20231013-095705/slices"
list_record_dir = sorted(os.listdir(root_dir))

In [13]:
list_record_dir[125]

'20231013-095705-00.49.45-00.50.00@Sogun'